<a href="https://colab.research.google.com/github/abuwildanm/Text-Mining/blob/master/Sentiment_Analysis_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis on Comments Regarding Jakarta's MRT Using the Naïve Bayes Algorithm

## Persiapan

In [0]:
# Import Library Standard
import numpy as np
import pandas as pd

# Import Library Sklearn
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold, train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Import Library untuk Stemming
!pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

     |████████████████████████████████| 215kB 2.7MB/s 


In [0]:
# Read Dataset
data = pd.read_excel('https://github.com/abuwildanm/Text-Mining/blob/master/Dataset%20MRT%20Jakarta.xlsx?raw=true')
# data = pd.read_excel('Dataset MRT Jakarta.xlsx')
data.head()

,Komentar,Sumber,Pakar 1 (Firda),Pakar 2 (Adel),Pakar 3 (Faiz),Pakar 4 (Marsha),Positif,Negatif,Hasill akhir
0,"tidak perlu menunggu lama utk menunggu MRT,krn...",Facebook,Positif,Positif,Positif,Positif,4,0,Positif
1,"alhamdulillah baru sempet naik MRt. nyaman, be...",Facebook,Positif,Positif,Positif,Positif,4,0,Positif
2,"Betul, petugas MRT Jakarta sangat ramah terhad...",Twitter,Positif,Positif,Positif,Positif,4,0,Positif
3,Hidup gw jadi lebih nyaman karena gak mikir ma...,Twitter,Positif,Positif,Positif,Positif,4,0,Positif
4,MRT adalah moda raya transportasi yang cepat d...,Twitter,Positif,Positif,Positif,Positif,4,0,Positif


In [0]:
df = data[['Komentar', 'Hasill akhir']].copy()
df.rename(columns={'Komentar':'komentar', 'Hasill akhir':'label'}, inplace=True)
df.head()

,komentar,label
0,"tidak perlu menunggu lama utk menunggu MRT,krn...",Positif
1,"alhamdulillah baru sempet naik MRt. nyaman, be...",Positif
2,"Betul, petugas MRT Jakarta sangat ramah terhad...",Positif
3,Hidup gw jadi lebih nyaman karena gak mikir ma...,Positif
4,MRT adalah moda raya transportasi yang cepat d...,Positif


In [0]:
print('Ukuran Dataset: ', df.shape)
n_document = df.shape[0]

Ukuran Dataset:  (30, 2)


## Preprocessing

### Spell Correction

### Stemming

In [0]:
# Membuat stemmer
stemmerFactory = StemmerFactory()
stemmer = stemmerFactory.create_stemmer()

# Daftar Stopword
stopWordRemoverFactory = StopWordRemoverFactory()
stopwords = stopWordRemoverFactory.get_stop_words()

In [0]:
# Proses Stem
for row in range(n_document):
  df.loc[row, 'komentar'] = stemmer.stem(df.loc[row, 'komentar'])

In [0]:
df

,komentar,label
0,tidak perlu tunggu lama utk tunggu mrt krna ad...,Positif
1,alhamdulillah baru sempet naik mrt nyaman bers...,Positif
2,betul tugas mrt jakarta sangat ramah hadap san...,Positif
3,hidup gw jadi lebih nyaman karena gak mikir ma...,Positif
4,mrt adalah moda raya transportasi yang cepat d...,Positif
5,nyobain moda transportasi baru yaitu mrtjkt te...,Positif
6,asik akhir nyobain mrtjakarta juga terima kasi...,Positif
7,dari ratu plaza ke blok m secepet lo masak mie...,Positif
8,terima kasih mrt jakarta atas layan baik nya m...,Positif
9,cobain mrt stasiun luas banget kereta juga ker...,Positif


## Perhitungan Bobot

In [0]:
vectorizer = CountVectorizer(stop_words=stopwords)
tf = vectorizer.fit_transform(df['komentar'])
tf = pd.DataFrame(tf.toarray(), columns=vectorizer.get_feature_names())
tf

,10,12,20an,2x,40,ac,admin,ain,aja,akhir,akses,alhamdulillah,alias,anak,anti,antri,apa,arah,aroma,asik,atas,atm,atur,baca,badan,bagus,baharu,bahasa,baik,balik,banget,bangun,bank,banyak,bapak,barat,baru,bau,bawa,bayar,...,tau,tdk,teladan,telat,tempat,terima,terimakasih,terus,tetap,tgl,the,thx,tiap,toilet,topup,transportasi,tugas,tuh,tuju,tunggu,tutup,twitter,uang,udara,udh,ulang,untul,up,usul,utk,waktu,warga,warna,wc,wkwk,work,wujud,yaa,yah,yg
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
6,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0


In [0]:
print('Fitur (Term): ', vectorizer.get_feature_names())

Fitur (Term):  ['10', '12', '20an', '2x', '40', 'ac', 'admin', 'ain', 'aja', 'akhir', 'akses', 'alhamdulillah', 'alias', 'anak', 'anti', 'antri', 'apa', 'arah', 'aroma', 'asik', 'atas', 'atm', 'atur', 'baca', 'badan', 'bagus', 'baharu', 'bahasa', 'baik', 'balik', 'banget', 'bangun', 'bank', 'banyak', 'bapak', 'barat', 'baru', 'bau', 'bawa', 'bayar', 'bbrp', 'bebas', 'bekal', 'bendung', 'berangkat', 'berapa', 'bersih', 'betul', 'biar', 'bijak', 'bikin', 'blok', 'bni', 'bnr2', 'brp', 'buat', 'bugar', 'buka', 'bundar', 'cari', 'cat', 'cek', 'celaka', 'cepat', 'cepet', 'cium', 'cobain', 'contoh', 'cukup', 'dah', 'dar', 'debet', 'deh', 'depan', 'dgn', 'dicapture', 'dingin', 'disabilitas', 'dlm', 'dong', 'dr', 'duduk', 'dukuh', 'dukung', 'duluan', 'ekspresi', 'elektronik', 'emg', 'emosi', 'empat', 'fasilitas', 'fatmawati', 'foto', 'ga', 'gak', 'ganti', 'gate', 'gerbong', 'gimana', 'gimna', 'gin', 'gitu', 'gmn', 'good', 'gorong', 'guys', 'gw', 'hadap', 'haji', 'hambat', 'hari', 'hasil', 'hati

In [0]:
tf_manual = tf.iloc[[2, 4, 6, 15, 27, 28, 1]]
feature_manual = (tf_manual != 0).any()
tf_manual = tf_manual.loc[:, feature_manual]
tf_manual

,akhir,alhamdulillah,anti,asik,atas,banget,banyak,baru,berangkat,bersih,betul,brp,buat,cari,celaka,cepat,dar,dingin,disabilitas,dukuh,dukung,fasilitas,hadap,hello,jadwal,jaga,jakarta,jempol,kantor,kasih,kerja,kok,kruk,kurang,lift,macet,makin,menit,moda,moga,...,mrt,mrtjakarta,mrtjkt,naik,nyaman,nyobain,pake,peduli,pergi,perintah,prioritas,proyek,pulang,ramah,raya,sama,sampah,sandang,sangat,saran,selama2nya,sempet,semua,skrng,smp,sponsor,stasiun,susah,telat,tempat,terima,terus,tetap,tiap,transportasi,tugas,untul,utk,waktu,yg
2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,1,1,0,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,...,2,0,0,1,1,0,1,0,1,0,0,0,1,1,0,0,0,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
6,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,...,1,1,0,0,1,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,1
15,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,...,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
27,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0
28,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0
1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [0]:
tf_manual.to_excel('TF Manual.xlsx')

## Pembagian Data

In [0]:
# Pembagian data train dan test
x_train, x_test, y_train, y_test = train_test_split(tf, df['label'], test_size=(10/n_document), stratify=df['label'])
tf_train = pd.DataFrame(x_train, columns=tf.columns)
tf_train['label'] = y_train
tf_test = pd.DataFrame(x_test, columns=tf.columns)
tf_test['label'] = y_test

In [0]:
tf_train

,10,12,20an,2x,40,ac,admin,ain,aja,akhir,akses,alhamdulillah,alias,anak,anti,antri,apa,arah,aroma,asik,atas,atm,atur,baca,badan,bagus,baharu,bahasa,baik,balik,banget,bangun,bank,banyak,bapak,barat,baru,bau,bawa,bayar,...,tdk,teladan,telat,tempat,terima,terimakasih,terus,tetap,tgl,the,thx,tiap,toilet,topup,transportasi,tugas,tuh,tuju,tunggu,tutup,twitter,uang,udara,udh,ulang,untul,up,usul,utk,waktu,warga,warna,wc,wkwk,work,wujud,yaa,yah,yg,label
6,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,Positif
10,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,Positif
12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Positif
21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,...,0,0,0,2,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Negatif
14,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Positif
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,Positif
18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Negatif
23,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Negatif
17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Negatif
20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Negatif


In [0]:
tf_train['label'].value_counts()

Negatif    10
Positif    10
Name: label, dtype: int64

In [0]:
tf_test

,10,12,20an,2x,40,ac,admin,ain,aja,akhir,akses,alhamdulillah,alias,anak,anti,antri,apa,arah,aroma,asik,atas,atm,atur,baca,badan,bagus,baharu,bahasa,baik,balik,banget,bangun,bank,banyak,bapak,barat,baru,bau,bawa,bayar,...,tdk,teladan,telat,tempat,terima,terimakasih,terus,tetap,tgl,the,thx,tiap,toilet,topup,transportasi,tugas,tuh,tuju,tunggu,tutup,twitter,uang,udara,udh,ulang,untul,up,usul,utk,waktu,warga,warna,wc,wkwk,work,wujud,yaa,yah,yg,label
15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,Negatif
29,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,...,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Negatif
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,Positif
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,Positif
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,Positif
22,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2,Negatif
19,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Negatif
13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Positif
27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Negatif
1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Positif


In [0]:
tf_test['label'].value_counts()

Positif    5
Negatif    5
Name: label, dtype: int64

In [0]:
# Ukuran Data Training & Testing
print('Ukuran data train:', tf_train.shape)
print('Ukuran data test:', tf_test.shape)

Ukuran data train: (20, 365)
Ukuran data test: (10, 365)


## Klasifikasi

### Benchmark

In [0]:
mnb = MultinomialNB()
model = mnb.fit(tf_train.iloc[:, :-1], tf_train.iloc[:, -1])
model.predict(tf_test.iloc[:, :-1])

array(['Negatif', 'Negatif', 'Positif', 'Positif', 'Negatif', 'Positif',
       'Negatif', 'Positif', 'Negatif', 'Positif'], dtype='<U7')

In [0]:
mnb.score(tf_test.iloc[:, :-1], tf_test.iloc[:, -1])

0.8

### Scratch

In [0]:
class MultinomialNaiveBayes():
  
  def __init__(self, alpha=1.0):
    """Inisialisasi class (constructor)
    
    Keyword Arguments:
        alpha {float} -- smoothing parameter (Laplacian/Lidstone) (default: {1.0})
    """

    self.prior = []
    self.likelihood = []
    self.posterior = []
    self.alpha = alpha

  def calculatePrior(self, label):
    """Untuk menghitung probabilitas kelas (prior)
    
    Arguments:
        label {pandas series} -- label pada data training
    
    Returns:
        pandas series -- nilai probabilitas prior
    """

    # prior = np.log(label.value_counts()) - np.log(label.size)
    prior = label.value_counts() / label.size
    return prior

  def calculateLikelihood(self, data, label):
    """Untuk menghitung probabilitas fitur pada suatu kelas (likelihood)
    
    Arguments:
        data {pandas dataframe} -- data training
        label {pandas series} -- label pada data training
    
    Returns:
        pandas dataframe -- nilai probabilitas likelihood
    """
    
    # Menggabungkan data dengan label
    sparse_data = pd.concat([data, label], axis=1)
    # Menghitung jumlah kemunculan suatu kata pada masing-masing kategori
    countWordLabel = sparse_data.groupby('label').sum()
    # Menghitung jumlah total kemunculan semua kata pada masing-masing kategori
    countAllWordLabel = countWordLabel.sum(axis=1)

    # likelihood = np.log((countWordLabel + self.alpha)).sub(np.log(countAllWordLabel + data.shape[1]), axis=0)
    likelihood = (countWordLabel + self.alpha).div(countAllWordLabel + data.shape[1], axis=0)
    return likelihood

  # # Option 1
  # def calculatePosterior(self, data):
  #   posterior = []
  #   for i in range(data.shape[0]):
  #     idx_term_test = (data.iloc[i] != 0)
  #     term_test = data.columns[idx_term_test]
  #     feature_test = np.intersect1d(term_test, self.likelihood.columns)
  #     posterior.append(self.prior * self.likelihood[feature_test].prod(axis=1))
    
  #   posterior = pd.DataFrame(posterior)
  #   return posterior

  def calculatePosterior(self, row_data):
    """Untuk menghitung probabilitas suatu kelas yang dipengaruhi oleh fitur-fitur yang ada (posterior)
    
    Arguments:
        row_data {pandas series} -- satu row data
    
    Returns:
        pandas dataframe -- nilai probabilitas posterior
    """

    # Mencari index term pada data testing
    idx_term_test = (row_data != 0)
    # Term yang muncul pada data test
    term_test = row_data.index[idx_term_test]
    # Term yang muncul pada data test dan data train
    feature_test = np.intersect1d(term_test, self.likelihood.columns)

    # posterior = self.prior + self.likelihood[feature_test].sum(axis=1)
    posterior = self.prior * self.likelihood[feature_test].prod(axis=1)
    return posterior

  def fit(self, x, y):
    """Untuk melakukan training model
    
    Arguments:
        x {pandas dataframe} -- data training
        y {pandas series} -- label pada data training
    """

    # Menghitung prior
    self.prior = self.calculatePrior(y)
    # Menghitung likelihood
    self.likelihood = self.calculateLikelihood(x, y)

  def predict(self, x):
    """Untuk melakukan testing model
    
    Arguments:
        x {pandas dataframe} -- data testing
    
    Returns:
        pandas series -- label hasil prediksi
    """

    # Menghitung posterior
    self.posterior = x.apply(self.calculatePosterior, axis=1)
    # Memprediksi label
    pred = self.posterior.idxmax(axis=1)

    return pred

  def score(self, x, y):
    """Untuk melakukan evaluasi model
    
    Arguments:
        x {pandas dataframe} -- data testing
        y {pandas series} -- label yang sebenarnya pada data testing
    
    Returns:
        float -- hasil evaluasi model
    """
    
    # Memprediksi label
    pred = self.predict(x)
    # Menghitung akurasi
    accuracy = accuracy_score(y, pred)

    return accuracy

In [0]:
mnb = MultinomialNaiveBayes()
mnb.fit(tf_train.iloc[:, :-1], tf_train.iloc[:, -1])
mnb.predict(tf_test.iloc[:, :-1])

15    Negatif
29    Negatif
2     Positif
0     Positif
3     Positif
22    Positif
19    Negatif
13    Positif
27    Negatif
1     Positif
dtype: object

In [0]:
mnb.score(tf_test.iloc[:, :-1], tf_test.iloc[:, -1])

0.9

In [0]:
mnb.prior

Negatif    0.5
Positif    0.5
Name: label, dtype: float64

In [0]:
mnb.likelihood

,10,12,20an,2x,40,ac,admin,ain,aja,akhir,akses,alhamdulillah,alias,anak,anti,antri,apa,arah,aroma,asik,atas,atm,atur,baca,badan,bagus,baharu,bahasa,baik,balik,banget,bangun,bank,banyak,bapak,barat,baru,bau,bawa,bayar,...,tau,tdk,teladan,telat,tempat,terima,terimakasih,terus,tetap,tgl,the,thx,tiap,toilet,topup,transportasi,tugas,tuh,tuju,tunggu,tutup,twitter,uang,udara,udh,ulang,untul,up,usul,utk,waktu,warga,warna,wc,wkwk,work,wujud,yaa,yah,yg
label,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Negatif,0.001701,0.003401,0.001701,0.003401,0.001701,0.003401,0.001701,0.003401,0.001701,0.003401,0.001701,0.001701,0.001701,0.003401,0.001701,0.005102,0.001701,0.003401,0.001701,0.001701,0.003401,0.006803,0.001701,0.001701,0.003401,0.001701,0.001701,0.001701,0.001701,0.001701,0.003401,0.001701,0.003401,0.006803,0.001701,0.001701,0.001701,0.003401,0.001701,0.005102,...,0.003401,0.001701,0.003401,0.003401,0.005102,0.001701,0.003401,0.003401,0.001701,0.003401,0.001701,0.003401,0.008503,0.005102,0.003401,0.001701,0.006803,0.003401,0.001701,0.001701,0.001701,0.001701,0.005102,0.001701,0.003401,0.006803,0.001701,0.001701,0.003401,0.001701,0.001701,0.001701,0.001701,0.003401,0.001701,0.001701,0.001701,0.001701,0.003401,0.005102
Positif,0.001838,0.001838,0.001838,0.001838,0.001838,0.001838,0.003676,0.001838,0.003676,0.005515,0.001838,0.001838,0.001838,0.001838,0.003676,0.001838,0.001838,0.001838,0.001838,0.003676,0.003676,0.001838,0.003676,0.003676,0.001838,0.005515,0.003676,0.003676,0.005515,0.001838,0.003676,0.003676,0.001838,0.001838,0.001838,0.003676,0.003676,0.001838,0.003676,0.001838,...,0.001838,0.001838,0.001838,0.001838,0.001838,0.005515,0.005515,0.003676,0.003676,0.001838,0.003676,0.001838,0.001838,0.001838,0.001838,0.005515,0.001838,0.001838,0.003676,0.001838,0.001838,0.003676,0.001838,0.001838,0.001838,0.001838,0.001838,0.003676,0.001838,0.003676,0.001838,0.003676,0.003676,0.001838,0.003676,0.003676,0.003676,0.001838,0.001838,0.003676


In [0]:
mnb.posterior

,Negatif,Positif
15,2.925911e-36,8.048879e-38
29,1.364326e-104,6.394366e-105
2,1.003318e-56,2.597494e-55
0,2.986868e-35,2.134563e-34
3,1.365059e-58,2.518578e-58
22,2.148236e-125,1.920829e-123
19,1.007188e-65,2.118512e-67
13,2.459450e-09,3.105799e-09
27,1.185090e-17,1.844085e-18
1,1.194747e-35,2.276867e-33


## Cross Validation

In [0]:
skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(tf, df['label'])

5

In [0]:
mnb = MultinomialNaiveBayes()

for train_index, test_index in skf.split(tf, df['label']):
  print("TRAIN:", train_index, "TEST:", test_index)
  mnb.fit(tf.iloc[train_index], df['label'].iloc[train_index])
  mnb.predict(tf.iloc[test_index])
  print("Score: ", mnb.score(tf.iloc[test_index], df['label'].iloc[test_index]))

TRAIN: [ 3  4  5  6  7  8  9 10 11 12 13 14 18 19 20 21 22 23 24 25 26 27 28 29] TEST: [ 0  1  2 15 16 17]
Score:  0.8333333333333334
TRAIN: [ 0  1  2  6  7  8  9 10 11 12 13 14 15 16 17 21 22 23 24 25 26 27 28 29] TEST: [ 3  4  5 18 19 20]
Score:  0.8333333333333334
TRAIN: [ 0  1  2  3  4  5  9 10 11 12 13 14 15 16 17 18 19 20 24 25 26 27 28 29] TEST: [ 6  7  8 21 22 23]
Score:  0.6666666666666666
TRAIN: [ 0  1  2  3  4  5  6  7  8 12 13 14 15 16 17 18 19 20 21 22 23 27 28 29] TEST: [ 9 10 11 24 25 26]
Score:  0.8333333333333334
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 15 16 17 18 19 20 21 22 23 24 25 26] TEST: [12 13 14 27 28 29]
Score:  0.8333333333333334
